In [1]:
import os
import pandas as pd

def xlsx_to_csv(file_name):
    save_name = os.path.splitext(file_name)[0]
    data_xls = pd.read_excel(file_name, index_col=0)
    save_file = r'{}.csv'.format(save_name)
    data_xls.to_csv(save_file, encoding='utf-8')
    return save_file

filename = '示例 - 超市.xls'
xlsx_to_csv(filename)

'示例 - 超市.csv'

In [2]:
import numpy as np  # 科学计算工具包
import pandas as pd  # 数据分析工具包
import matplotlib.pyplot as plt # 图表绘制工具包
import seaborn as sns # 基于 matplot, 导入 seaborn 会修改默认的 matplotlib 配色方案和绘图样式，这会提高图表的可读性和美观性
import os,pymysql
from sqlalchemy import create_engine # 数据库

plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei')  # 解决Seaborn中文显示问题

# 在 jupyter notebook 里面显示图表
%matplotlib inline 

In [5]:
orders = pd.read_csv("示例 - 超市.csv")
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9959 entries, 0 to 9958
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   行 ID    9959 non-null   int64  
 1   订单 ID   9959 non-null   object 
 2   订单日期    9959 non-null   object 
 3   发货日期    9959 non-null   object 
 4   邮寄方式    9959 non-null   object 
 5   客户 ID   9959 non-null   object 
 6   客户名称    9959 non-null   object 
 7   细分      9959 non-null   object 
 8   城市      9959 non-null   object 
 9   省/自治区   9959 non-null   object 
 10  国家      9959 non-null   object 
 11  地区      9959 non-null   object 
 12  产品 ID   9959 non-null   object 
 13  类别      9959 non-null   object 
 14  子类别     9959 non-null   object 
 15  产品名称    9959 non-null   object 
 16  销售额     9959 non-null   float64
 17  数量      9959 non-null   int64  
 18  折扣      9959 non-null   float64
 19  利润      9959 non-null   float64
dtypes: float64(3), int64(2), object(15)
memory usage: 1.5+ MB


In [6]:
# 日期时间数据类型转化的函数

def transform_datetime(data,column_list):
    # func: 日期时间字符串转化函数
    # data: dataframe类型
    # num: 数字类型，显示前几行数据
    # return: 日期时间类型数据
    
    for i in column_list:
        data[i] = pd.to_datetime(data[i])
    print('日期时间数据类型转化完成')
    
data = orders
column_list =['订单日期'] 
transform_datetime(data,column_list)

日期时间数据类型转化完成


In [13]:
# 新增字段：购买年月
orders['下单年月'] = orders['订单日期'].dt.to_period('M')

# 客户首次购买时间
new_df = orders.groupby('客户 ID',as_index=False).下单年月.agg([min])


new_df

,min
客户 ID,
丁君-15280,2015-03
丁妮-18610,2015-04
丁娇-14695,2016-02
丁婵-10990,2016-03
丁崆-15535,2015-01
...,...
龙婷-21115,2015-04
龙廷-21295,2015-05
龙谦-19300,2015-05


In [14]:
# 合并数据集

basic = pd.merge(orders,new_df, on='客户 ID',how='left')
basic

,行 ID,订单 ID,订单日期,发货日期,邮寄方式,客户 ID,客户名称,细分,城市,省/自治区,...,产品 ID,类别,子类别,产品名称,销售额,数量,折扣,利润,下单年月,min
0,1,US-2018-1357144,2018-04-27,2018-04-29,二级,曾惠-14485,曾惠,公司,杭州,浙江,...,办公用-用品-10002717,办公用品,用品,"Fiskars 剪刀, 蓝色",129.696,2,0.4,-60.704,2018-04,2015-11
1,2,CN-2018-1973789,2018-06-15,2018-06-19,标准级,许安-10165,许安,消费者,内江,四川,...,办公用-信封-10004832,办公用品,信封,"GlobeWeis 搭扣信封, 红色",125.440,2,0.0,42.560,2018-06,2016-11
2,3,CN-2018-1973789,2018-06-15,2018-06-19,标准级,许安-10165,许安,消费者,内江,四川,...,办公用-装订-10001505,办公用品,装订机,"Cardinal 孔加固材料, 回收",31.920,2,0.4,4.200,2018-06,2016-11
3,4,US-2018-3017568,2018-12-09,2018-12-13,标准级,宋良-17170,宋良,公司,镇江,江苏,...,办公用-用品-10003746,办公用品,用品,"Kleencut 开信刀, 工业",321.216,4,0.4,-27.104,2018-12,2015-09
4,5,CN-2017-2975416,2017-05-31,2017-06-02,二级,万兰-15730,万兰,消费者,汕头,广东,...,办公用-器具-10003452,办公用品,器具,"KitchenAid 搅拌机, 黑色",1375.920,3,0.0,550.200,2017-05,2015-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9954,9996,CN-2018-4318875,2018-05-28,2018-06-02,标准级,巩光-13435,巩光,消费者,义乌,浙江,...,技术-配件-10003923,技术,配件,"SanDisk 记忆卡, 实惠",944.244,3,0.4,-314.916,2018-05,2015-05
9955,9997,CN-2018-4318875,2018-05-28,2018-06-02,标准级,巩光-13435,巩光,消费者,义乌,浙江,...,办公用-收纳-10003383,办公用品,收纳具,"Eldon 盘, 蓝色",447.720,2,0.0,35.560,2018-05,2015-05
9956,9998,CN-2018-4318875,2018-05-28,2018-06-02,标准级,巩光-13435,巩光,消费者,义乌,浙江,...,家具-椅子-10002448,家具,椅子,"SAFCO 折叠椅, 红色",239.988,1,0.4,-88.032,2018-05,2015-05
9957,9999,CN-2018-4318875,2018-05-28,2018-06-02,标准级,巩光-13435,巩光,消费者,义乌,浙江,...,技术-配件-10002808,技术,配件,"罗技 路由器, 耐用",4851.588,7,0.4,-1617.392,2018-05,2015-05


In [15]:
basic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9959 entries, 0 to 9958
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   行 ID    9959 non-null   int64         
 1   订单 ID   9959 non-null   object        
 2   订单日期    9959 non-null   datetime64[ns]
 3   发货日期    9959 non-null   object        
 4   邮寄方式    9959 non-null   object        
 5   客户 ID   9959 non-null   object        
 6   客户名称    9959 non-null   object        
 7   细分      9959 non-null   object        
 8   城市      9959 non-null   object        
 9   省/自治区   9959 non-null   object        
 10  国家      9959 non-null   object        
 11  地区      9959 non-null   object        
 12  产品 ID   9959 non-null   object        
 13  类别      9959 non-null   object        
 14  子类别     9959 non-null   object        
 15  产品名称    9959 non-null   object        
 16  销售额     9959 non-null   float64       
 17  数量      9959 non-null   int64         
 18  折扣      

In [17]:
# 按照年月判断新老用户的函数

def status(data):
    if data['下单年月'] > data['min']:
        tag = '老客户'
    else:
        tag = '新客户'
    return tag

basic['tag']=basic.apply(status,axis=1)
basic

,行 ID,订单 ID,订单日期,发货日期,邮寄方式,客户 ID,客户名称,细分,城市,省/自治区,...,类别,子类别,产品名称,销售额,数量,折扣,利润,下单年月,min,tag
0,1,US-2018-1357144,2018-04-27,2018-04-29,二级,曾惠-14485,曾惠,公司,杭州,浙江,...,办公用品,用品,"Fiskars 剪刀, 蓝色",129.696,2,0.4,-60.704,2018-04,2015-11,老客户
1,2,CN-2018-1973789,2018-06-15,2018-06-19,标准级,许安-10165,许安,消费者,内江,四川,...,办公用品,信封,"GlobeWeis 搭扣信封, 红色",125.440,2,0.0,42.560,2018-06,2016-11,老客户
2,3,CN-2018-1973789,2018-06-15,2018-06-19,标准级,许安-10165,许安,消费者,内江,四川,...,办公用品,装订机,"Cardinal 孔加固材料, 回收",31.920,2,0.4,4.200,2018-06,2016-11,老客户
3,4,US-2018-3017568,2018-12-09,2018-12-13,标准级,宋良-17170,宋良,公司,镇江,江苏,...,办公用品,用品,"Kleencut 开信刀, 工业",321.216,4,0.4,-27.104,2018-12,2015-09,老客户
4,5,CN-2017-2975416,2017-05-31,2017-06-02,二级,万兰-15730,万兰,消费者,汕头,广东,...,办公用品,器具,"KitchenAid 搅拌机, 黑色",1375.920,3,0.0,550.200,2017-05,2015-11,老客户
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9954,9996,CN-2018-4318875,2018-05-28,2018-06-02,标准级,巩光-13435,巩光,消费者,义乌,浙江,...,技术,配件,"SanDisk 记忆卡, 实惠",944.244,3,0.4,-314.916,2018-05,2015-05,老客户
9955,9997,CN-2018-4318875,2018-05-28,2018-06-02,标准级,巩光-13435,巩光,消费者,义乌,浙江,...,办公用品,收纳具,"Eldon 盘, 蓝色",447.720,2,0.0,35.560,2018-05,2015-05,老客户
9956,9998,CN-2018-4318875,2018-05-28,2018-06-02,标准级,巩光-13435,巩光,消费者,义乌,浙江,...,家具,椅子,"SAFCO 折叠椅, 红色",239.988,1,0.4,-88.032,2018-05,2015-05,老客户
9957,9999,CN-2018-4318875,2018-05-28,2018-06-02,标准级,巩光-13435,巩光,消费者,义乌,浙江,...,技术,配件,"罗技 路由器, 耐用",4851.588,7,0.4,-1617.392,2018-05,2015-05,老客户


In [19]:
basic.columns.values

array(['行 ID', '订单 ID', '订单日期', '发货日期', '邮寄方式', '客户 ID', '客户名称', '细分',
       '城市', '省/自治区', '国家', '地区', '产品 ID', '类别', '子类别', '产品名称', '销售额',
       '数量', '折扣', '利润', '下单年月', 'min', 'tag'], dtype=object)

In [21]:
# 导出到 mysql 数据库的函数

def export_mysql(data,user,password,host_port,db,table_name):
    # func: 导出到 mysql 数据库函数
    # data: Dataframe 类型
    # user: 用户名
    # password: 密码
    # host_port: 主机，端口
    # db: 数据库名
    # table_name: 表名

    print('将清洗后的数据导出到 mysql')
    engine = create_engine("mysql+pymysql://{}:{}@{}/{}".format(user, password, host_port, db))
    con = engine.connect()#创建连接
    data.to_sql(table_name,engine,if_exists='replace',index=False)
    print('成功导出')
    
user = 'root'
password = '1234567890'
host_port = '127.0.0.1:3306'
db = 'test'

data = basic[['行 ID', '订单 ID', '订单日期', '发货日期', '邮寄方式', '客户 ID', '客户名称', '细分',
       '城市', '省/自治区', '国家', '地区', '产品 ID', '类别', '子类别', '产品名称', '销售额',
       '数量', '折扣', '利润', 'tag']]
table_name = 'shop_basic'

export_mysql(data,user,password,host_port,db,table_name)

将清洗后的数据导出到 mysql
成功导出


In [22]:
basic[basic['订单 ID'] == 'CN-2015-2686699']

,行 ID,订单 ID,订单日期,发货日期,邮寄方式,客户 ID,客户名称,细分,城市,省/自治区,...,类别,子类别,产品名称,销售额,数量,折扣,利润,下单年月,min,tag
275,278,CN-2015-2686699,2015-11-25,2015-11-30,二级,程德-12040,程德,小型企业,宜春,江西,...,家具,用具,"Tenex 相框, 一包多件",1211.000,5,0.0,363.300,2015-11,2015-09,老客户
276,279,CN-2015-2686699,2015-11-25,2015-11-30,二级,程德-12040,程德,小型企业,宜春,江西,...,技术,配件,"Memorex 路由器, 可编程",2311.960,2,0.0,346.640,2015-11,2015-09,老客户
3791,3812,CN-2015-2686699,2015-02-20,2015-02-25,标准级,丁蔓-18025,丁蔓楚,消费者,苏州,江苏,...,办公用品,装订机,"Ibico 装订机, 耐用",190.176,4,0.4,-15.904,2015-02,2015-02,新客户
3792,3813,CN-2015-2686699,2015-02-20,2015-02-25,标准级,丁蔓-18025,丁蔓楚,消费者,苏州,江苏,...,办公用品,信封,"Cameo 局间信封, 银色",697.620,3,0.0,230.160,2015-02,2015-02,新客户
3793,3814,CN-2015-2686699,2015-02-20,2015-02-25,标准级,丁蔓-18025,丁蔓楚,消费者,苏州,江苏,...,技术,设备,"冲电器 喷墨打印机, 多色",879.480,1,0.4,-322.560,2015-02,2015-02,新客户
3794,3815,CN-2015-2686699,2015-02-20,2015-02-25,标准级,丁蔓-18025,丁蔓楚,消费者,苏州,江苏,...,办公用品,美术,"Binney & Smith 钢笔, 整包",68.376,6,0.8,-215.544,2015-02,2015-02,新客户
3795,3816,CN-2015-2686699,2015-02-20,2015-02-25,标准级,丁蔓-18025,丁蔓楚,消费者,苏州,江苏,...,家具,椅子,"SAFCO 凳子, 可调",1431.612,3,0.4,-334.068,2015-02,2015-02,新客户
5759,5783,CN-2015-2686699,2015-02-02,2015-02-08,标准级,林康-15400,林康,公司,莱阳,山东,...,技术,复印机,"佳能 传真复印机, 红色",3538.080,4,0.0,565.600,2015-02,2015-01,老客户


In [ ]:
按照